# Capstone EDA & Baseline — Policyholder Lapse (Churn Proxy)
*Last updated:* 2025-08-24 22:48

**Research question:** Can syllabus‑aligned ML classifiers (baseline Logistic Regression) predict which policyholders are likely to lapse within 12 months and support actionable **risk tiers** for retention?

**Dataset (no login required):** **UCI ML Repository — Telco Customer Churn**  
https://archive.ics.uci.edu/ml/machine-learning-databases/00470/WA_Fn-UseC_-Telco-Customer-Churn.csv

> This notebook is designed to satisfy **Module 20.1** requirements: data cleaning, EDA/visualizations, feature engineering, baseline model + evaluation, and an executive brief.


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import (roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay, brier_score_loss)
from sklearn.calibration import calibration_curve

import warnings
warnings.filterwarnings('ignore')

sns.set_context('talk')
plt.rcParams['figure.figsize'] = (8,5)
pd.set_option('display.max_columns', 100)

## Load Data (UCI — direct CSV)

In [ ]:
import pandas as pd

UCI_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00470/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(UCI_URL)

print("Shape:", df.shape)
df.head(3)

## Basic Structure & Info

In [ ]:
rows, cols = df.shape
print(f'Rows: {rows}, Columns: {cols}')
df.info()

## Data Cleaning

In [ ]:
# Convert TotalCharges to numeric (blanks to NaN), impute from MonthlyCharges if missing
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    if 'MonthlyCharges' in df.columns:
        df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'])

# Drop duplicate customers if any
if 'customerID' in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=['customerID'])
    print('Removed duplicates:', before - len(df))

# Standardize target labels
df['Churn'] = df['Churn'].astype(str).str.strip().str.title()  # 'Yes'/'No'

## Target Distribution

In [ ]:
y = (df['Churn'] == 'Yes').astype(int)
ax = y.value_counts().sort_index().plot(kind='bar')
ax.set_xticklabels(['No','Yes'])
ax.set_title('Lapse (Churn) Distribution')
ax.set_xlabel('Churn'); ax.set_ylabel('Count')
plt.show()
print('Churn rate:', y.mean().round(4))

## Exploratory Data Analysis (EDA)

In [ ]:
import seaborn as sns

# Numeric distributions split by churn
num_preview = [c for c in ['tenure','MonthlyCharges','TotalCharges'] if c in df.columns]
fig, axes = plt.subplots(1, len(num_preview), figsize=(6*len(num_preview), 5))
if len(num_preview) == 1: axes = [axes]
for ax, col in zip(axes, num_preview):
    sns.histplot(data=df, x=col, hue='Churn', kde=True, stat='density', common_norm=False, ax=ax)
    ax.set_title(f'{col} by Churn'); ax.set_xlabel(col); ax.set_ylabel('Density')
plt.tight_layout(); plt.show()

# Categorical stacked bars vs churn
for col in [c for c in ['Contract','PaymentMethod','PaperlessBilling','InternetService'] if c in df.columns]:
    ct = (pd.crosstab(df[col], df['Churn'], normalize='index')*100).round(1)
    ct.plot(kind='bar', stacked=True)
    plt.title(f'{col} vs Churn (row %)'); plt.ylabel('Percentage'); plt.legend(title='Churn')
    plt.tight_layout(); plt.show()

# Boxplots for outliers
for c in [col for col in ['MonthlyCharges','TotalCharges','tenure'] if col in df.columns]:
    df.boxplot(column=c, by='Churn', grid=False)
    plt.suptitle(''); plt.title(f'Boxplot of {c} by Churn'); plt.xlabel('Churn'); plt.ylabel(c)
    plt.show()

# Correlation heatmap (numeric)
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
if len(num_cols) > 1:
    corr = df[num_cols].corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', square=True, cbar=True)
    plt.title('Correlation Heatmap (Numeric)')
    plt.tight_layout(); plt.show()

## Outlier Analysis (IQR & Z-score)

In [ ]:
import numpy as np, pandas as pd

def outlier_summary(series):
    s = pd.to_numeric(series, errors='coerce').dropna()
    if len(s) == 0:
        return {'count': 0, 'iqr_outliers': 0, 'iqr_pct': 0.0, 'z_outliers': 0, 'z_pct': 0.0}
    q1, q3 = np.percentile(s, [25, 75]); iqr = q3 - q1
    iqr_low, iqr_high = q1 - 1.5*iqr, q3 + 1.5*iqr
    iqr_mask = (s < iqr_low) | (s > iqr_high)
    mu, sd = s.mean(), s.std(ddof=0); z_mask = (np.abs((s - mu) / (sd if sd>0 else 1)) > 3)
    return {'count': int(s.shape[0]), 'iqr_outliers': int(iqr_mask.sum()), 'iqr_pct': float(iqr_mask.mean()*100),
            'z_outliers': int(z_mask.sum()), 'z_pct': float(z_mask.mean()*100)}

numeric_focus = [c for c in ['MonthlyCharges','TotalCharges','tenure'] if c in df.columns]
outlier_report = {c: outlier_summary(df[c]) for c in numeric_focus}
pd.DataFrame(outlier_report).T

## Feature Engineering

In [ ]:
# Service count
service_cols = [c for c in ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies'] if c in df.columns]
df['ServiceCount'] = df[service_cols].apply(lambda r: sum(x=='Yes' for x in r), axis=1) if service_cols else 0

# Tenure band
if 'tenure' in df.columns:
    bins = [0, 6, 12, 24, 48, 72, 1000]
    labels = ['0-6','6-12','12-24','24-48','48-72','72+']
    df['tenure_band'] = pd.cut(df['tenure'], bins=bins, labels=labels, right=False)

# AutoPay flag
if 'PaymentMethod' in df.columns:
    df['AutoPay'] = df['PaymentMethod'].str.contains('automatic', case=False, na=False).map({True:'Yes', False:'No'})

# ChargesRatio
if set(['MonthlyCharges','TotalCharges','tenure']).issubset(df.columns):
    denom = (df['tenure'].clip(lower=1) * df['MonthlyCharges']).replace(0, np.nan)
    df['ChargesRatio'] = (pd.to_numeric(df['TotalCharges'], errors='coerce') / denom).replace([np.inf, -np.inf], np.nan).fillna(1.0)

# Interaction
if 'MonthlyCharges' in df.columns and 'Contract' in df.columns:
    high_m = df['MonthlyCharges'] > df['MonthlyCharges'].median()
    mtm = df['Contract'].eq('Month-to-month')
    df['HighMonthly_MTM'] = np.where(high_m & mtm, 'Yes', 'No')

df.head(3)

## Evaluation Metric Rationale
- **Primary:** ROC‑AUC (robust to class imbalance; measures ranking for **risk tiers**).
- **Secondary:** Accuracy, Precision, Recall, F1 at default and tuned thresholds.
- **Calibration:** Brier score + calibration curve to ensure probability quality.


## Baseline Model — Logistic Regression

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Features & target
y = (df['Churn'] == 'Yes').astype(int)
X = df.drop(columns=['Churn','customerID'], errors='ignore')
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()

preprocess = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

log_reg = Pipeline([('prep', preprocess),
                    ('clf', LogisticRegression(max_iter=500, solver='liblinear'))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7, stratify=y)
log_reg.fit(X_train, y_train)
proba = log_reg.predict_proba(X_test)[:,1]
pred05 = (proba >= 0.5).astype(int)

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
metrics_baseline = {
    'ROC_AUC': roc_auc_score(y_test, proba),
    'Accuracy': accuracy_score(y_test, pred05),
    'Precision': precision_score(y_test, pred05),
    'Recall': recall_score(y_test, pred05),
    'F1': f1_score(y_test, pred05)
}
metrics_baseline

## Calibration & Threshold Tuning

In [ ]:
# Calibration
frac_pos, mean_pred = calibration_curve(y_test, proba, n_bins=10, strategy='quantile')
brier = brier_score_loss(y_test, proba)

# Threshold tuning (Youden's J)
from sklearn.metrics import roc_curve
fpr, tpr, thr = roc_curve(y_test, proba)
youden = tpr - fpr
best_idx = np.argmax(youden)
best_thr = thr[best_idx]
pred_tuned = (proba >= best_thr).astype(int)

def metric_row(label, y_true, y_pred, y_prob):
    return {'Label': label,
            'ROC_AUC': roc_auc_score(y_true, y_prob),
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred)}

import pandas as pd
metrics_df = pd.DataFrame([
    metric_row('0.5 threshold', y_test, pred05, proba),
    metric_row(f'Tuned (YoudenJ={best_thr:.3f})', y_test, pred_tuned, proba)
])
metrics_df

In [ ]:
# Diagnostics: ROC, PR, Calibration, Confusion matrices
fig, axes = plt.subplots(1,3, figsize=(18,5))
RocCurveDisplay.from_predictions(y_test, proba, ax=axes[0]); axes[0].set_title('ROC — Logistic Regression')
PrecisionRecallDisplay.from_predictions(y_test, proba, ax=axes[1]); axes[1].set_title('PR — Logistic Regression')
axes[2].plot(mean_pred, frac_pos, marker='o', label='Model'); axes[2].plot([0,1],[0,1],'--', label='Perfect')
axes[2].set_title(f'Calibration (Brier={brier:.3f})'); axes[2].set_xlabel('Mean predicted'); axes[2].set_ylabel('Fraction positive'); axes[2].legend()
plt.tight_layout(); plt.show()

import seaborn as sns
from sklearn.metrics import confusion_matrix
fig, axes = plt.subplots(1,2, figsize=(12,5))
sns.heatmap(confusion_matrix(y_test, pred05), annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Confusion Matrix @ 0.5'); axes[0].set_xlabel('Predicted'); axes[0].set_ylabel('Actual')
sns.heatmap(confusion_matrix(y_test, pred_tuned), annot=True, fmt='d', cmap='Greens', ax=axes[1])
axes[1].set_title(f'Confusion Matrix @ Tuned ({best_thr:.3f})'); axes[1].set_xlabel('Predicted'); axes[1].set_ylabel('Actual')
plt.tight_layout(); plt.show()

## (Optional) Model Comparison (Syllabus‑Aligned)

In [ ]:
# Compare models by 5-fold CV ROC-AUC (kept optional for rubric; logistic remains baseline)
models = {
    'LogReg': LogisticRegression(max_iter=500, solver='liblinear'),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=300, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'SVM_RBF': SVC(kernel='rbf', probability=True, random_state=42)
}

preprocess = ColumnTransformer([('num', StandardScaler(), num_cols),
                                ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}
for name, clf in models.items():
    pipe = Pipeline([('prep', preprocess), ('clf', clf)])
    aucs = cross_val_score(pipe, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
    cv_results[name] = {'ROC_AUC_mean': aucs.mean(), 'ROC_AUC_std': aucs.std()}

pd.DataFrame(cv_results).T.sort_values('ROC_AUC_mean', ascending=False)

## Executive Brief (Initial Findings — Paste into README/Word)

**Problem**  
Lapses (policy cancellations) erode profitability: acquisition costs are sunk while expected premiums vanish. We predicted lapse risk to prioritize retention outreach.

**Data & Method**  
We used the public Telco Churn dataset (UCI). Baseline **Logistic Regression** (syllabus) predicts lapse probability; we validated via train/test split, calibration, and threshold tuning, focusing on **ROC‑AUC** due to class imbalance.

**Key Drivers**  
Short tenure, month‑to‑month contracts, electronic check payment, higher monthly charges, and fewer add‑on services increase lapse risk (directionally consistent with industry intuition).

**Business Impact**  
Ranking customers by probability allows **risk tiers** (e.g., top 10–20%) for targeted retention campaigns. Threshold tuning balances catching more lapsers vs. minimizing unnecessary outreach.

**Next Actions**  
Pilot retention offers for high‑risk cohorts; evaluate lift; consider ensemble models and cost‑aware thresholds in the next module.


## Export Key Results to README.md

In [ ]:
# After running the notebook, run this cell to append a Results section to your README.md
from pathlib import Path

readme = Path('README.md')
lines = []
lines.append('## Results (Run on UCI Telco Churn)\n')
lines.append(f'- Baseline: **Logistic Regression**\n')
lines.append(f'- Test metrics @0.5 → AUC: {metrics_baseline["ROC_AUC"]:.3f}, Acc: {metrics_baseline["Accuracy"]:.3f}, '
             f'Prec: {metrics_baseline["Precision"]:.3f}, Rec: {metrics_baseline["Recall"]:.3f}, F1: {metrics_baseline["F1"]:.3f}\n')
lines.append(f'- Tuned threshold ({best_thr:.3f}) → AUC: {metrics_df.iloc[1]["ROC_AUC"]:.3f}, Acc: {metrics_df.iloc[1]["Accuracy"]:.3f}, '
             f'Prec: {metrics_df.iloc[1]["Precision"]:.3f}, Rec: {metrics_df.iloc[1]["Recall"]:.3f}, F1: {metrics_df.iloc[1]["F1"]:.3f}\n')
lines.append(f'\nCalibration (Brier): **{brier:.3f}**\n\n')

if readme.exists():
    prev = readme.read_text(encoding='utf-8')
else:
    prev = ''

readme.write_text(prev + ('\n' if not prev.endswith('\n') else '') + ''.join(lines), encoding='utf-8')
print('README.md updated.')

## Rubric Checklist (Module 20.1)
- **Project organization:** Clear headings, explanatory markdown; README with summary, file links.
- **Syntax & code quality:** Clean imports, comments, sensible variables; avoid massive printouts.
- **Visualizations:** Numeric (hist/KDE), categorical (stacked bar), boxplots, correlation heatmap; readable titles/labels.
- **Data Cleaning & EDA:** Convert/impute `TotalCharges`; drop duplicates; **outlier analysis**; feature engineering (`tenure_band`, `AutoPay`, `ChargesRatio`, `ServiceCount`, interaction).
- **Modeling:** Baseline **Logistic Regression**; ROC‑AUC rationale; precision/recall/F1; calibration + threshold tuning; confusion matrices.
